In [1]:
import gensim
import os
import numpy as np
import nltk
from keras.models import Sequential
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers import Activation, TimeDistributed, Dense, RepeatVector, recurrent, Embedding
from keras.preprocessing.sequence import pad_sequences
import gensim
import json

c:\users\gavin0228\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\gavin0228\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_lines(file_path, separator, name_idx, content_idx, start_line = 0, limit = 10000):
    ln = 1
    prev_name = None
    sentences = []
    for line in open(file_path, 'r', encoding="utf-8"):
        if ln >= limit:
            break
        elif ln < start_line:
            ln += 1
            continue
        ln += 1
        items = line.split(separator)
        name = items[name_idx].lower()
        content = items[content_idx].lower().replace('?', '').replace('!', '').replace( '.', '').replace( ',', '').replace( '-', '')
        words = content.split()
        if prev_name != name:
            sentences.append(words)
        else:
            #print(prev_name, "spoke again")
            sentences[-1].extend([" "] + words )
        prev_name = name
    return sentences



sentences = read_lines("movie_lines.txt", " +++$+++ ", 3, 4, 0, 1000)


In [3]:
SENTENCE_LENGTH = 20
VOCAB_SIZE = 5000

In [4]:
#flatten_sentences = [word for sentence in sentences for word in sentence ]



In [5]:
def build_vocab_dict(sentences, padding_char, unknown_char):
    word_freq = nltk.FreqDist(np.hstack(sentences))
    word_freq.pop(padding_char, None)
    word_freq.pop(unknown_char, None)
    vocab = word_freq.most_common(VOCAB_SIZE - 2)

    vocab.insert(0, (padding_char,1))
    vocab.append( (unknown_char,1))
    
    vocab_dict = {pair[0]: id for id, pair in enumerate(vocab)}
    
    idx_dict = {idx:word for word, idx in vocab_dict.items()}
    print(idx_dict)
    return vocab_dict, idx_dict

vocab_dict, idx_dict = build_vocab_dict(sentences, ' ', 'UNK')
print(vocab_dict, idx_dict)

{0: ' ', 1: 'you', 2: 'i', 3: 'to', 4: 'the', 5: 'a', 6: 'of', 7: 'me', 8: 'and', 9: 'that', 10: 'is', 11: 'what', 12: 'your', 13: 'it', 14: 'my', 15: 'not', 16: 'with', 17: 'have', 18: 'for', 19: 'do', 20: 'know', 21: 'be', 22: 'we', 23: 'are', 24: "don't", 25: 'he', 26: 'no', 27: 'in', 28: 'this', 29: "i'm", 30: 'go', 31: 'like', 32: 'so', 33: "you're", 34: 'but', 35: 'out', 36: 'on', 37: 'just', 38: 'her', 39: 'think', 40: 'was', 41: 'want', 42: 'get', 43: 'about', 44: 'all', 45: 'can', 46: 'she', 47: 'who', 48: 'then', 49: 'never', 50: "it's", 51: 'how', 52: 'if', 53: 'say', 54: 'him', 55: 'up', 56: 'one', 57: 'tell', 58: "that's", 59: 'did', 60: 'at', 61: "can't", 62: 'they', 63: 'why', 64: 'will', 65: 'them', 66: 'god', 67: 'when', 68: 'got', 69: 'there', 70: 'right', 71: 'would', 72: 'yeah', 73: 'need', 74: 'an', 75: 'here', 76: 'as', 77: 'see', 78: "he's", 79: 'were', 80: 'man', 81: 'something', 82: 'or', 83: 'going', 84: 'because', 85: 'let', 86: 'could', 87: 'has', 88: 'had',

In [6]:
def save_vocab(vocab_dict):
    f = open("vocab_dict.json", 'w') 
    f.write(json.dumps(vocab_dict)) 
    f.close() 
def load_vocab():
    f = open("vocab_dict.json", 'r') 
    vocab_dict = json.loads(f.read() )
    f.close() 
    idx_dict = idx_dict = {idx:word for word, idx in vocab_dict.items()}
    return vocab_dict, idx_dict

In [7]:
#save_vocab(vocab_dict)
vocab_dict, idx_dict = load_vocab()
#print(vocab_dict, idx_dict)

In [8]:
sentences=["hi","i'm good","hello","hello","hey","hey there", 'shit','i know right']
sentences= sentences + sentences + sentences + sentences
sentences= sentences + sentences + sentences + sentences
sentences= sentences + sentences + sentences + sentences

In [10]:
def sentence_to_vec(sentences, vocab_dict, unknown_char, sentence_length):
    l = len(sentences)
    vec = []
    unk_idx = vocab_dict[unknown_char]

    for sen in sentences:
        vec.append( [vocab_dict[x] if x in vocab_dict else unk_idx for x in sen ][:20])

    padded = pad_sequences(vec, maxlen=sentence_length, dtype='int32')
    return padded

In [11]:
def to_one_hot(vec, sentence_length, vocab_length):
    print((len(vec), sentence_length, vocab_length))
    res = np.zeros((len(vec), sentence_length, vocab_length))
    for i, sen in enumerate(vec):
        for j, num in enumerate(sen):
            res[i, j, num] = 1
    return res

x_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 0]
y_sentences = [sentences[i] for i in range(len(sentences)) if i % 2 == 1]

x_vec = sentence_to_vec(x_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
y_vec = sentence_to_vec(y_sentences, vocab_dict, 'UNK', SENTENCE_LENGTH)
y_vec = to_one_hot(y_vec, SENTENCE_LENGTH, VOCAB_SIZE)

print(x_vec.shape, y_vec.shape)

(256, 20, 5000)
(256, 20) (256, 20, 5000)


In [12]:
def create_model(x_vocab_len, x_max_len, y_vocab_len, y_max_len, hidden_size, num_layers):
    model = Sequential()

    # Creating encoder network
    model.add(Embedding(x_vocab_len, 1024, input_length=x_max_len, mask_zero=True))
    model.add(LSTM(hidden_size))
    model.add(RepeatVector(y_max_len))

    # Creating decoder network
    for _ in range(num_layers):
        model.add(LSTM(hidden_size, return_sequences=True))
    model.add(TimeDistributed(Dense(y_vocab_len)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy',
            optimizer='rmsprop',
            metrics=['accuracy'])
    return model
model = create_model(VOCAB_SIZE, SENTENCE_LENGTH, VOCAB_SIZE, SENTENCE_LENGTH, 1024, 3)
#model.summary()

In [13]:
model.load_weights("new_chatbot_model_2.h5")


In [15]:
model.fit(x_vec, y_vec, batch_size=50, nb_epoch=10)
model.save_weights("new_chatbot_model_2.h5")

c:\users\gavin0228\appdata\local\programs\python\python36\lib\site-packages\keras\models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
256/256 [==============================] - 69s 269ms/step - loss: 3.4276 - acc: 0.7875
Epoch 2/10
256/256 [==============================] - 69s 269ms/step - loss: 3.4267 - acc: 0.7875
Epoch 3/10
256/256 [==============================] - 68s 266ms/step - loss: 3.4261 - acc: 0.7875
Epoch 4/10
256/256 [==============================] - 68s 267ms/step - loss: 3.4258 - acc: 0.7875
Epoch 5/10
256/256 [==============================] - 68s 268ms/step - loss: 3.4256 - acc: 0.7875
Epoch 6/10
256/256 [==============================] - 68s 266ms/step - loss: 3.4254 - acc: 0.7875
Epoch 7/10
256/256 [==============================] - 68s 268ms/step - loss: 3.4253 - acc: 0.7875
Epoch 8/10
256/256 [==============================] - 69s 268ms/step - loss: 3.4253 - acc: 0.7875
Epoch 9/10
256/256 [==============================] - 68s 267ms/step - loss: 3.4252 - acc: 0.7875
Epoch 10/10
256/256 [==============================] - 70s 274ms/step - loss: 3.4252 - acc: 0.7875


In [19]:
sen = "cat"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '')
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

print(res)
#print(res.shape)
vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0]])

[[[9.9998689e-01 4.0827668e-11 9.7582520e-10 ... 3.8952983e-11
   3.6537703e-11 2.2677855e-06]
  [9.9998760e-01 3.6697249e-11 2.3979516e-09 ... 3.8478706e-11
   3.8124691e-11 6.9342555e-06]
  [9.9999607e-01 3.0284553e-12 4.4108039e-10 ... 3.0431861e-12
   2.4943487e-12 3.2734831e-06]
  ...
  [7.0232491e-05 4.1751147e-10 1.6487901e-10 ... 3.4267786e-10
   2.4199961e-10 9.9990952e-01]
  [6.9350303e-05 3.7213099e-10 1.4376458e-10 ... 3.0860983e-10
   2.1804344e-10 9.9991000e-01]
  [7.9637342e-05 5.3672544e-10 2.1689969e-10 ... 4.3419446e-10
   3.2343367e-10 9.9989593e-01]]]


'                    UNK UNK UNK UNK   UNK UNK UNK UNK UNK'

In [20]:
sen = "fake"
sen = sen.lower().replace('?', '').replace('!', '').replace( '.', '')
vec = sentence_to_vec([sen], vocab_dict, 'UNK', SENTENCE_LENGTH)
#print(model.predict(vec).shape)
res = model.predict(vec)

print(res)
#print(res.shape)
vec_y = np.argmax(res, axis=2)
" ".join([idx_dict[x] for x in vec_y[0]])

[[[9.9999857e-01 3.2589224e-12 9.3922065e-11 ... 3.1314458e-12
   2.5074296e-12 4.8688878e-07]
  [9.9999666e-01 8.6134936e-12 7.7180751e-10 ... 8.9240629e-12
   8.3433260e-12 2.4668311e-06]
  [9.9999785e-01 4.8040408e-12 9.2584684e-10 ... 5.1005489e-12
   4.6339447e-12 1.9709955e-06]
  ...
  [5.2986514e-05 1.2316465e-10 4.3413100e-11 ... 1.1484347e-10
   6.2179040e-11 9.9993908e-01]
  [5.0652576e-05 1.2210231e-10 4.3271178e-11 ... 1.1575066e-10
   6.2303836e-11 9.9994147e-01]
  [6.3117077e-05 2.0384563e-10 6.9741664e-11 ... 1.9073049e-10
   1.0341887e-10 9.9992716e-01]]]


'                    UNK UNK UNK UNK   UNK UNK UNK UNK UNK'